# Data science in Microsoft Fabric

I worked with the diabetes dataset from the Azure Open Datasets. After loading the data, I converted the data to a Pandas dataframe: a common structure for working with data in rows and columns. 
Since this is my first time running any Spark code in this session, the Spark pool must be started. This means that the first run in the session took about 7 minutes to complete. Subsequent runs were quicker.

In [1]:
# Azure storage access info for open dataset diabetes
blob_account_name = "azureopendatastorage"
blob_container_name = "mlsamples"
blob_relative_path = "diabetes"
blob_sas_token = r"" # Blank since container is Anonymous access
    
# Set Spark config to access  blob storage
wasbs_path = f"wasbs://%s@%s.blob.core.windows.net/%s" % (blob_container_name, blob_account_name, blob_relative_path)
spark.conf.set("fs.azure.sas.%s.%s.blob.core.windows.net" % (blob_container_name, blob_account_name), blob_sas_token)
print("Remote blob path: " + wasbs_path)
    
# Spark read parquet, note that it won't load any data yet by now
df = spark.read.parquet(wasbs_path)

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 3, Finished, Available, Finished)

Remote blob path: wasbs://mlsamples@azureopendatastorage.blob.core.windows.net/diabetes


1. Data exploration

In [2]:
display(df)

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 010a8d7f-ea2a-487a-8da3-ab9d36e383e2)

2. Data Preparation

In [3]:
df = df.toPandas()
df.head()

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 5, Finished, Available, Finished)

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y
0,59,2,32.1,101.0,157,93.2,38.0,4.0,4.8598,87,151
1,48,1,21.6,87.0,183,103.2,70.0,3.0,3.8918,69,75
2,72,2,30.5,93.0,156,93.6,41.0,4.0,4.6728,85,141
3,24,1,25.3,84.0,198,131.4,40.0,5.0,4.8903,89,206
4,50,1,23.0,101.0,192,125.4,52.0,4.0,4.2905,80,135


3. Data wrangling

In [4]:
# Code generated by Data Wrangler for pandas DataFrame

def clean_data(df):
    # Created column 'Risk' from formula
    df['Risk'] = (df['Y'] > 211.5).astype(int)
    return df

df_clean = clean_data(df.copy())
display(df_clean)

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 13, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, e0bff51c-ac08-46d1-93ad-87d503a29bad)

In [13]:
df_clean.describe()

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 22, Finished, Available, Finished)

,AGE,SEX,BMI,BP,S1,S2,S3,S4,S5,S6,Y,Risk
count,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000,442.000000
mean,48.518100,1.468326,26.375792,94.647014,189.140271,115.439140,49.788462,4.070249,4.641411,91.260181,152.133484,0.251131
std,13.109028,0.499561,4.418122,13.831283,34.608052,30.413081,12.934202,1.290450,0.522391,11.496335,77.093005,0.434155
min,19.000000,1.000000,18.000000,62.000000,97.000000,41.600000,22.000000,2.000000,3.258100,58.000000,25.000000,0.000000
25%,38.250000,1.000000,23.200000,84.000000,164.250000,96.050000,40.250000,3.000000,4.276700,83.250000,87.000000,0.000000
50%,50.000000,1.000000,25.700000,93.000000,186.000000,113.000000,48.000000,4.000000,4.620050,91.000000,140.500000,0.000000
75%,59.000000,2.000000,29.275000,105.000000,209.750000,134.500000,57.750000,5.000000,4.997200,98.000000,211.500000,0.750000
max,79.000000,2.000000,42.200000,133.000000,301.000000,242.400000,99.000000,9.090000,6.107000,124.000000,346.000000,1.000000


### Training machine learning models
1. Training a regression model

In [6]:
from sklearn.model_selection import train_test_split
    
X, y = df_clean[['AGE','SEX','BMI','BP','S1','S2','S3','S4','S5','S6']].values, df_clean['Y'].values
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 15, Finished, Available, Finished)

I created an MLflow experiment named diabetes-regression. My models will be tracked in this experiment

In [7]:
import mlflow
experiment_name = "diabetes-regression"
mlflow.set_experiment(experiment_name)

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 16, Finished, Available, Finished)

2025/07/17 19:17:20 INFO mlflow.tracking.fluent: Experiment with name 'diabetes-regression' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1752779845015, experiment_id='1d5aa204-3255-4438-a2d7-ec8152acaf22', last_update_time=None, lifecycle_stage='active', name='diabetes-regression', tags={}>

The code below trains a regression model using Linear Regression. Parameters, metrics, and artifacts, are automatically logged with MLflow.

In [9]:
from sklearn.linear_model import LinearRegression
    
with mlflow.start_run():
   mlflow.autolog()
    
   model = LinearRegression()
   model.fit(X_train, y_train)

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 18, Finished, Available, Finished)

2025/07/17 19:18:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/07/17 19:18:25 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/trusted-service-user/cluster-env/trident_env/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


2. Train a classification model

I ran the following code below to split the data into a training and test dataset, and to separate the features from the label Risk I want to predict

In [10]:
from sklearn.model_selection import train_test_split
    
X, y = df_clean[['AGE','SEX','BMI','BP','S1','S2','S3','S4','S5','S6']].values, df_clean['Risk'].values
    
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 19, Finished, Available, Finished)

The code below creates an MLflow experiment named diabetes-classification. My models will be tracked in this experiment.

In [11]:
import mlflow
experiment_name = "diabetes-classification"
mlflow.set_experiment(experiment_name)

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 20, Finished, Available, Finished)

2025/07/17 19:20:02 INFO mlflow.tracking.fluent: Experiment with name 'diabetes-classification' does not exist. Creating a new experiment.


<Experiment: artifact_location='', creation_time=1752780003543, experiment_id='69d07bf1-8933-4e99-8654-890690815407', last_update_time=None, lifecycle_stage='active', name='diabetes-classification', tags={}>

The code below trains a classification model using Logistic Regression. Parameters, metrics, and artifacts, are automatically logged with MLflow.

In [12]:
from sklearn.linear_model import LogisticRegression
    
with mlflow.start_run():
    mlflow.sklearn.autolog()

    model = LogisticRegression(C=1/0.1, solver="liblinear").fit(X_train, y_train)

StatementMeta(, 23642f29-0b83-4bb4-9487-19e3329919d3, 21, Finished, Available, Finished)